## 1. Importar Librerias

In [64]:
import pandas as pd
import numpy as np
import warnings

## 2. Cargar Datos Crudos

In [65]:
df = pd.read_csv('../data/raw/medicamentos_raw.csv')
print(f"Datos cargados: {df.shape[0]} filas, {df.shape[1]} columnas")

Datos cargados: 12534 filas, 9 columnas


In [66]:
df.head()

,principio_activo,unidad_de_dispensacion,concentracion,unidad_base,nombre_comercial,fabricante,precio_por_tableta,factoresprecio,numerofactor
0,Midazolam,Ampolla,Midazolam 15 mg,ml,Dormicum,Siegfried,11199.800000,Alto,3
1,Acido Valproico,Tableta,Divalproato Sodico 500 mg,mg,Valcote,Lafrancol,3752.866667,Medio,2
2,Acido Valproico,Tableta,Divalproato Sodico 500 mg,mg,Valcote,Lafrancol,1777.266522,Medio,2
3,Fluoxetina,Capsula,Fluoxetina 20 mg,mg,Fluoxetina,Genfar,329.295281,Medio,2
4,Proximetacaina,Frasco,Proximetacaina 5 mg,ml,Alcaine,Alcon,64184.745760,Medio,2


## 3. Normalizar Nombres de Columnas

In [67]:
df.columns = df.columns.str.lower().str.strip()
print("Columnas normalizadas")
print(df.columns.tolist())

Columnas normalizadas
['principio_activo', 'unidad_de_dispensacion', 'concentracion', 'unidad_base', 'nombre_comercial', 'fabricante', 'precio_por_tableta', 'factoresprecio', 'numerofactor']


## 4. Convertir Tipos de Datos

In [68]:
if 'precio_por_tableta' in df.columns:
    df['precio_por_tableta'] = pd.to_numeric(df['precio_por_tableta'], errors='coerce')
    print("Precio convertido a numerico")

if 'numerofactor' in df.columns:
    df['numerofactor'] = pd.to_numeric(df['numerofactor'], errors='coerce')
    print("Numerofactor convertido a numerico")

Precio convertido a numerico
Numerofactor convertido a numerico


## 5. Valores Nulos

In [69]:
print("Valores nulos por columna:")
print(df.isnull().sum())

Valores nulos por columna:
principio_activo          0
unidad_de_dispensacion    0
concentracion             0
unidad_base               0
nombre_comercial          0
fabricante                0
precio_por_tableta        0
factoresprecio            0
numerofactor              0
dtype: int64


In [70]:
filas_antes = len(df)
df = df.dropna(subset=['precio_por_tableta'])
print(f"Eliminadas {filas_antes - len(df)} filas con precio nulo")

Eliminadas 0 filas con precio nulo


In [71]:
columnas_texto = ['fabricante', 'principio_activo', 'nombre_comercial', 'unidad_de_dispensacion']

for col in columnas_texto:
    if col in df.columns:
        df[col].fillna('No especificado', inplace=True) # Rellenar valores nulos con 'No especificado'
        print(f"Columna {col} completada")

Columna fabricante completada
Columna principio_activo completada
Columna nombre_comercial completada
Columna unidad_de_dispensacion completada


/tmp/ipykernel_4258/3218767031.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna('No especificado', inplace=True) # Rellenar valores nulos con 'No especificado'


## 6. Eliminar Duplicados

In [72]:
duplicados = df.duplicated().sum() # Contar duplicados
df = df.drop_duplicates() # Eliminar duplicados
print(f"Eliminados {duplicados} duplicados")

Eliminados 126 duplicados


## 7. Detectar Valores Atípicos

In [73]:
q1 = df['precio_por_tableta'].quantile(0.25) # Cuartil 1
q3 = df['precio_por_tableta'].quantile(0.75) # Cuartil 3
iqr = q3 - q1 # Rango intercuartil

limite_inferior = q1 - 1.5 * iqr 
limite_superior = q3 + 1.5 * iqr

print(f"IQR: ${iqr:.2f}")
print(f"Limite inferior: ${limite_inferior:.2f}")
print(f"Limite superior: ${limite_superior:.2f}")

IQR: $16849.97
Limite inferior: $-24267.19
Limite superior: $43132.68


In [74]:
valores_atipicos= df[(df['precio_por_tableta'] < limite_inferior) | 
              (df['precio_por_tableta'] > limite_superior)]
print(f"valores atipicos detectados: {len(valores_atipicos)} ({len(valores_atipicos )/len(df)*100:.2f}%)")

valores atipicos detectados: 1964 (15.83%)


## 8. Filtrar Datos Extremos

In [75]:
filas_antes = len(df)
df = df[df['precio_por_tableta'] > 0]
print(f"Eliminadas {filas_antes - len(df)} filas con precio <= 0")

Eliminadas 0 filas con precio <= 0


In [76]:
percentil_99 = df['precio_por_tableta'].quantile(0.99)
filas_antes = len(df)
df = df[df['precio_por_tableta'] <= percentil_99]
print(f"Eliminadas {filas_antes - len(df)} filas extremas")

Eliminadas 125 filas extremas


## 9. Normalizar Textos

In [77]:
columnas_normalizar = ['principio_activo', 'fabricante', 'nombre_comercial']

for col in columnas_normalizar:
    if col in df.columns:
        df[col] = df[col].str.strip().str.title() # Normalizar texto (quitar espacios y poner mayuscula inicial)
        print(f"Columna {col} normalizada")

Columna principio_activo normalizada
Columna fabricante normalizada
Columna nombre_comercial normalizada


## 10. Filtrar Medicamentos Orales

In [78]:
if 'unidad_de_dispensacion' in df.columns:
    print("Unidades de dispensacion:")
    print(df['unidad_de_dispensacion'].value_counts()) # Contar ocurrencias de cada unidad
    
    filas_antes = len(df)
    df = df[df['unidad_de_dispensacion'].str.contains('Tableta|Capsula', case=False, na=False)] # Filtrar solo medicamentos orales
    print(f"\nFiltrado a medicamentos orales: {len(df)} registros") 
    print(f"Eliminados: {filas_antes - len(df)} registros")

Unidades de dispensacion:
unidad_de_dispensacion
Tableta                  4948
Frasco                   2448
Capsula                  1227
Vial                      984
Tubo                      698
Ampolla                   645
Bolsa                     310
Jeringa Prellenada        246
Sobre                     244
Tableta Masticable         81
Inhalador                  68
Ovulo                      66
Spray                      56
Cilindro                   49
Parche                     39
Aerosol                    37
Botella                    23
Pluma                      23
Sachet                     17
Supositorio                15
Tableta Vaginal            11
Implante Intrauterino      10
Termo Criogenico            8
Tableta Efervescente        7
Cartucho                    6
Anillo Vaginal              5
Carpule                     3
Enema                       3
Capsula Vaginal             2
Envase                      2
Capsula Dura                1
Caja                 

## 11. Crear Categorias de Precio

In [79]:
def categorizar_precio(precio):
    if precio < 10000:
        return 'Economico'
    elif precio < 50000:
        return 'Moderado'
    elif precio < 200000:
        return 'Costoso'
    else:
        return 'Muy Costoso'

df['categoria_precio'] = df['precio_por_tableta'].apply(categorizar_precio)
print("Categorias creadas")
print(df['categoria_precio'].value_counts())

Categorias creadas
categoria_precio
Economico      5716
Moderado        337
Costoso         152
Muy Costoso      72
Name: count, dtype: int64


## 12. Resumen de Datos Limpios

In [80]:
print("RESUMEN DE DATOS LIMPIOS")

print(f"Total de registros: {len(df)}")
print(f"Total de columnas: {df.shape[1]}")
print(f"Valores nulos: {df.isnull().sum().sum()}")
print(f"Duplicados: {df.duplicated().sum()}")
print(f"Principios activos unicos: {df['principio_activo'].nunique()}")
print(f"Fabricantes unicos: {df['fabricante'].nunique()}")
print(f"Precio min: ${df['precio_por_tableta'].min():.2f}")
print(f"Precio max: ${df['precio_por_tableta'].max():.2f}")
print(f"Precio promedio: ${df['precio_por_tableta'].mean():.2f}")

RESUMEN DE DATOS LIMPIOS
Total de registros: 6277
Total de columnas: 10
Valores nulos: 0
Duplicados: 0
Principios activos unicos: 867
Fabricantes unicos: 272
Precio min: $0.09
Precio max: $1875982.30
Precio promedio: $11211.51


## 13. Guardar Datos Limpios

In [81]:
df.to_csv('../data/medicamentos_limpios.csv', index=False, encoding='utf-8-sig')
print(f"Datos guardados: {len(df)} registros")

Datos guardados: 6277 registros
